<a href="https://colab.research.google.com/github/dvdblk/hack4good-oecd/blob/main/hack4good-oecd/experiments/carlos/extract_docs_in_colab/extract_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain faiss-cpu openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

import re
import os
import openai
import tiktoken
from operator import itemgetter
import typing
import numpy as np
import pandas as pd
import copy
import time
import json

from topic import Topic, recursive_topic_creator
from util import get_standard_doc_splits
from query_handler import QueryHandler

In [4]:
OPENAI_API_KEY = "redacted"
# OPENAI_API_KEY = "redacted"
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [23]:
multiple_q_dict = {
    "questions" : ["summary", "binary"],
    "top" : {
        "summary": "Summarize whether and how {topic} is mentioned in the document.",
        "binary": "Is {topic} mentioned in the document?",
        "sentiment": "How is the sentiment towards {topic} in the document?"},
    "relation" :{
        "summary": "With regard to {topic}, summarize whether and how the document mentions {subtopic}.",
        "binary": "With regard to {topic}, does the document mention {subtopic}?",
        "sentiment": "With regard to {topic}, what is the sentiment of {subtopic} in the document?"},
    "link": {"basic": "With regard to {technology} does the document mention {skill_caps}?"},
    # policy, investment
    "formatting" : {
        "summary": " Use one short sentence.",
        "binary": " Answer 1 for yes, 0 for no.", # " Yes -> 1 or No -> 0"
        "sentiment": " Positive -> 1, Negative -> -1, or Neutral -> 0"} # " Answer 1 for positive, 0 for neutral and -1 for negative sentiment"
}

In [32]:
# RUN TO RESET PROGRESS
import importlib
import query_handler

# all_files_results = json.load(open("all_files_results_171123-1748_gpt35-1106.json", "r"))
all_files_results = {}

CHUNK_SIZE = 2000

df = pd.read_csv('topic_datasheet.csv')
topics = recursive_topic_creator(df)

In [33]:
# !unzip data.zip
file_lst = os.listdir("data")
folder = "data/"

In [34]:
importlib.reload(query_handler)

<module 'query_handler' from '/content/query_handler.py'>

In [ ]:
working_lst = [i for i in file_lst if i not in all_files_results.keys()]
print("len working_lst", len(working_lst), ", len file_lst", len(file_lst))

for f in working_lst:
    print("working on: ", f)
    splits = get_standard_doc_splits(folder+f, chunk_size=CHUNK_SIZE)
    vectorstore = FAISS.from_documents(splits, embedding=OpenAIEmbeddings())

    qh = query_handler.QueryHandler(
        topics,
        multiple_q_dict,
        vectorstore,
        rag_topk=8,
        model_name="gpt-3.5-turbo-1106", #"gpt-4-1106-preview",
        query_json_split_size=3,
        sleep_time=1)

    res = qh.run()
    res["chunk_size"] = CHUNK_SIZE
    res["additional_notes"] = ""

    all_files_results[f] = res
    print("#"*50)

len working_lst 24 , len file_lst 24
working on:  UK_99.txt
Working on Advanced Computing...
{'general_Summary': 'The document mentions advanced computing under the umbrella of digital technologies related to AI and automation.', 'general_Binary': 1, 'Quantum Computing_Summary': 'The document does not mention Quantum Computing in relation to Advanced Computing.', 'Quantum Computing_Binary': 0, 'High-Performance Computing (HPC)_Summary': 'The document does not specifically mention High-Performance Computing (HPC) in the context of Advanced Computing.', 'High-Performance Computing (HPC)_Binary': 0, 'Distributed Computing_Summary': 'The document does not mention Distributed Computing in relation to Advanced Computing.', 'Distributed Computing_Binary': 0, 'Cloud Computing_Summary': 'The document does not specifically mention Cloud Computing.', 'Cloud Computing_Binary': 0, 'Artificial Intelligence_Summary': "The document extensively discusses Artificial Intelligence, its impact on the labor

In [20]:
# TEST PROBLEMATICS
splits = get_standard_doc_splits("data/UK_03.txt", chunk_size=CHUNK_SIZE)
vectorstore = FAISS.from_documents(splits, embedding=OpenAIEmbeddings())

qh = query_handler.QueryHandler(
    topics,
    multiple_q_dict,
    vectorstore,
    rag_topk=8,
    model_name="gpt-3.5-turbo-1106", #"gpt-4-1106-preview",
    query_json_split_size=3,
    sleep_time=1)

res = qh.run()

Working on Advanced Computing...
debug: 
resp_dict:  dict_keys(['general_Summary', 'general_Binary', 'Quantum Computing_Summary', 'Quantum Computing_Binary', 'High-Performance Computing (HPC)_Summary', 'High-Performance Computing (HPC)_Binary', 'Distributed Computing_Summary', 'Distributed Computing_Binary']) , dict_keys:  dict_keys(['general_Summary', 'general_Binary', 'Quantum Computing_Summary', 'Quantum Computing_Binary', 'High-Performance Computing (HPC)_Summary', 'High-Performance Computing (HPC)_Binary', 'Distributed Computing_Summary', 'Distributed Computing_Binary'])
debug: 
resp_dict:  dict_keys(['general_Summary', 'general_Binary', 'Cloud Computing_Summary', 'Cloud Computing_Binary', 'Artificial Intelligence_Summary', 'Artificial Intelligence_Binary', 'Big Data Analytics_Summary', 'Big Data Analytics_Binary']) , dict_keys:  dict_keys(['general_Summary', 'general_Binary', 'Cloud Computing_Summary', 'Cloud Computing_Binary', 'Artificial Intelligence_Summary', 'Artificial Intel

In [ ]:
json.dump(all_files_results,open("all_files_results_181123-1312_gpt35-1106_all.json", "w"))

# Ask Advanced Questions

In [ ]:
specific_question = {
    "policy": {
        "summary": "Summarize policies, if any, the document recommends for {topic} in one sentence.",
        "binary": "Does the document recommend any policies for {topic}? Answer 1 for yes, 0 for no."
    },
    "investment": {
        "summary": "Summarize the investment, if any, the document recommends for {topic} in one sentence.",
        "binary" : "Does the document recommend investment for {topic}? Answer 1 for yes, 0 for no."
    }
}
# "sentiment": "For {topic} is the sentiment positive, negative or neutral? Output 1 for positive, -1 for negative, 0 for neutral."

In [ ]:
# RUN THIS TO RESET ADVANCED SEARCH
importlib.reload(query_handler)
extended_results = {}

In [ ]:
for key in all_files_results.keys():
    if key in extended_results.keys():
        continue
    print("working on: ", key)
    splits = get_standard_doc_splits(folder+key, chunk_size=CHUNK_SIZE)
    vectorstore = FAISS.from_documents(splits, embedding=OpenAIEmbeddings())

    qh = query_handler.QueryHandler(
        topics,
        multiple_q_dict,
        vectorstore,
        rag_topk=8,
        model_name="gpt-3.5-turbo-1106", #"gpt-4-1106-preview",
        query_json_split_size=3,
        sleep_time=0.1)

    extended_results[key] = qh.traverse_advanced(all_files_results[key]["content"], specific_question)

In [ ]:
json.dump(extended_results, open("extended_results_1811_upto85.json", "w"))
from google.colab import files
files.download("extended_results_1811_upto85.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# oh a new function
def json_to_dataframe(doc_dict):
    df = pd.DataFrame()
    for file_key in doc_dict.keys():
        print(file_key)
        file_dict = doc_dict[file_key]["content"]
        to_pd_dict = {}
        count = 0
        for topic in file_dict.keys():
            if isinstance(file_dict[topic], dict):
                my_dict = {}
                count += len(file_dict[topic].keys())
                print(len(file_dict[topic].keys()))
                for k in file_dict[topic].keys():
                    col_name = topic.title() + "_" + k.title()
                    col_name = col_name.replace("_", "")
                    col_name = col_name.replace(" ", "")
                    my_dict[col_name] = file_dict[topic][k]

                to_pd_dict.update(my_dict)

        print(count)
        to_pd_dict["doc_name"] = file_key
        df = pd.concat([df, pd.DataFrame([to_pd_dict])], ignore_index=True)


    cols = list(df.columns.values)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    return df

In [10]:
my_ex = json.load(open("extended_results_1811_upto85.json", "r"))

In [46]:
count = 0
filename = "UK_52.txt"
for key in my_ex[filename].keys():
    count += len(my_ex[filename][key].keys())
    print(key)
    for k in my_ex[filename][key]:
        # if not k in my_ex["UK_07.txt"][key].keys():
        print(k)
print(count)

Advanced Computing
general_Summary
general_Binary
Quantum Computing_Summary
Quantum Computing_Binary
High-Performance Computing (HPC)_Summary
High-Performance Computing (HPC)_Binary
Distributed Computing_Summary
Distributed Computing_Binary
Cloud Computing_Summary
Cloud Computing_Binary
Artificial Intelligence_Summary
Artificial Intelligence_Binary
Big Data Analytics_Summary
Big Data Analytics_Binary
FPGA Technology_Summary
FPGA Technology_Binary
Advanced Computing_policy_summary
Advanced Computing_policy_binary
Advanced Computing_investment_summary
Advanced Computing_investment_binary
Quantum Computing_policy_summary
Quantum Computing_policy_binary
Quantum Computing_investment_summary
Quantum Computing_investment_binary
High-Performance Computing (HPC)_policy_summary
High-Performance Computing (HPC)_policy_binary
High-Performance Computing (HPC)_investment_summary
High-Performance Computing (HPC)_investment_binary
Distributed Computing_policy_summary
Distributed Computing_policy_binar

In [ ]:
df = json_to_dataframe(my_ex)

In [ ]:
for k1, k2 in zip(my_ex["UK_07.txt"]["content"]["Clean Energy"].keys(), my_ex["UK_36.txt"]["content"]["Clean Energy"].keys()):
    if k1 != k2:
        print(k1, "###", k2)

Clean energy_integration_Binary ### Clean energy integration_Binary


In [ ]:
df.to_csv("extended_datasheet_upto85.csv")

In [ ]:
from google.colab import files
files.download("extended_datasheet_upto85.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
eldf = pd.read_csv("binary_datasheet_upto85.csv")

In [44]:
340+170

510